In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from scraper_wiki_main import parsuj_misje_z_url, dekompresuj_html, pobierz_soup
import json
import copy
from bs4 import BeautifulSoup
import zlib
import base64
from sqlalchemy import text
import re
import json

from moduly.db_core import utworz_engine_do_db

In [3]:
soup_nie_dziala = pobierz_soup("https://www.wowhead.com/beta/quest=92163/the-loa-of-murlocs") 

In [4]:
x = soup_nie_dziala.find(class_="page-controls").find_all_next()

In [5]:
def wyciagnij_strefe_z_wowhead(soup):
    skrypty = soup.find_all('script', type="text/javascript")
    
    for skrypt in skrypty:
        if skrypt.string and "new Mapper" in skrypt.string:
            dopasowanie = re.search(r'new Mapper\((\{.*?\})\);', skrypt.string)
            
            if dopasowanie:
                json_str = dopasowanie.group(1)
                try:
                    dane = json.loads(json_str)
                    if "objectives" in dane:
                        cele = dane["objectives"]
                        for id_celu, szczegoly in cele.items():
                            if "zone" in szczegoly:
                                return szczegoly["zone"]
                                
                except json.JSONDecodeError:
                    continue
                    
    return "Nie znaleziono strefy"

In [6]:
strefa = wyciagnij_strefe_z_wowhead(soup_nie_dziala)
print(f"Znaleziona strefa: {strefa}")

Znaleziona strefa: Nie znaleziono strefy


In [7]:
strefa2 = wyciagnij_strefe_z_wowhead(soup_nie_dziala)
print(f"Znaleziona strefa: {strefa2}")

Znaleziona strefa: Nie znaleziono strefy


In [8]:
def wyciagnij_strefe_z_storyline(soup):
    link_storyline = soup.find("a", class_="quick-facts-storyline-title")
    
    if link_storyline:
        pelny_tekst = link_storyline.get_text(strip=True)

        if " - " in pelny_tekst:
            strefa = pelny_tekst.split(" - ")[0]
            return strefa
        
        return pelny_tekst
        
    return "Nie znaleziono strefy w Storyline"

strefa = wyciagnij_strefe_z_storyline(soup_nie_dziala)
print(strefa)

The Loa of Murlocs


In [9]:
def wyciagnij_kraine_z_breadcrumb(soup: BeautifulSoup) -> str:
    breadcrumb = soup.find("nav", class_="breadcrumb")
    
    if breadcrumb:
        linki = breadcrumb.find_all("a")
        
        if linki:
            ostatni_link = linki[-1]
            return ostatni_link.get_text(strip=True)
            
    return ""

In [10]:
wyciagnij_kraine_z_breadcrumb(soup_nie_dziala)

''

In [11]:
def wyciagnij_kraine_z_breadcrumb_poprawione(soup):
    # 1. Znajdź główny kontener nawigacji
    nav = soup.find("nav", class_="breadcrumb")
    
    if nav:
        # 2. Znajdź WSZYSTKIE spany w środku, nieważne czy mają klasę "arrow" czy nie
        wszystkie_spany = nav.find_all("span")
        
        if wszystkie_spany:
            # 3. Weź OSTATNI span z listy
            ostatni_span = wszystkie_spany[-1]
            
            # 4. Wyciągnij z niego tekst (szukamy <a>, bo tekst jest w linku)
            link = ostatni_span.find("a")
            if link:
                return link.get_text(strip=True)
            else:
                # Jeśli nie ma linku, zwróć sam tekst ze spana
                return ostatni_span.get_text(strip=True)
                
    return ""

In [12]:
wyciagnij_kraine_z_breadcrumb_poprawione(soup_nie_dziala)

''

In [13]:
n = soup_nie_dziala.find("title")
if n:
    print("tak")
    print(n)
else:
    print("nie")

tak
<title>The Loa of Murlocs - Quest - 12.0.1 Beta</title>


In [14]:
def znajdz_gdzie_ukrywa_sie_tekst(soup, szukana_fraza):
    html_content = str(soup)
    print(f"--- SZUKAM FRAZY: '{szukana_fraza}' ---")
    
    if szukana_fraza not in html_content:
        print("ALARM: Tej frazy w ogóle nie ma w pobranym kodzie! Wowhead wysłał coś innego.")
        return
    
    start = 0
    licznik = 0
    while True:
        idx = html_content.find(szukana_fraza, start)
        if idx == -1:
            break
        
        licznik += 1
        kontekst_start = max(0, idx - 100)
        kontekst_koniec = min(len(html_content), idx + len(szukana_fraza) + 100)
        fragment = html_content[kontekst_start:kontekst_koniec]

        fragment = fragment.replace("\n", " ").replace("\r", "")
        
        print(f"\n[ZNALEZISKO #{licznik}]")
        print(f"...{fragment}...")
        
        start = idx + len(szukana_fraza)

In [15]:
znajdz_gdzie_ukrywa_sie_tekst(soup_nie_dziala, "Zul'Aman")

--- SZUKAM FRAZY: 'Zul'Aman' ---

[ZNALEZISKO #1]
...Quest - 12.0.1 Beta</title> <meta content="Investigate the Loa of Murlocs at Atal'Abasi. A level 83 Zul'Aman Quest. Rewards . Added in World of Warcraft: Midnight. Always up to date with the latest patch." na...

[ZNALEZISKO #2]
...property="twitter:domain"/> <meta content="Investigate the Loa of Murlocs at Atal'Abasi. A level 83 Zul'Aman Quest. Rewards . Added in World of Warcraft: Midnight. Always up to date with the latest patch." pr...

[ZNALEZISKO #3]
...rty="twitter:description"/> <meta content="Investigate the Loa of Murlocs at Atal'Abasi. A level 83 Zul'Aman Quest. Rewards . Added in World of Warcraft: Midnight. Always up to date with the latest patch." pr...

[ZNALEZISKO #4]
...urlocs" property="og:url"/> <meta content="Investigate the Loa of Murlocs at Atal'Abasi. A level 83 Zul'Aman Quest. Rewards . Added in World of Warcraft: Midnight. Always up to date with the latest patch." pr...

[ZNALEZISKO #5]
...e, Retail Database,

In [16]:
def wyciagnij_kraine_z_meta_opisow(soup: BeautifulSoup) -> str:
    tagi_do_sprawdzenia = [
        {"name": "description"},
        {"property": "og:description"},
        {"property": "twitter:description"}
    ]
    
    tekst_opisu = ""
    
    for atrybuty in tagi_do_sprawdzenia:
        meta = soup.find("meta", attrs=atrybuty)
        if meta:
            tekst_opisu = meta.get("content", "")
            if tekst_opisu:
                break
    
    if not tekst_opisu:
        return ""

    match = re.search(r"A level \d+ (.+?) Quest", tekst_opisu)
    
    if match:
        return match.group(1).strip()
        
    return ""

In [17]:
wyciagnij_kraine_z_meta_opisow(soup_nie_dziala)

"Zul'Aman"

In [18]:
def wyciagnij_tytul_wowhead(soup: BeautifulSoup) -> str:
    tytul = soup.find("h1", class_="heading-size-1")
    return tytul.get_text(strip=True) if tytul else ""

In [19]:
wyciagnij_tytul_wowhead(soup_nie_dziala)

'The Loa of Murlocs'